In [ ]:
# Se va a usar la libreria de pandasSQL para hacer las queries
# (Google Colab)
!pip install pandas pandasql


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=9c0a09eb629ee596c7b1dbb497a2e0abcf212f06bc7c2dfda1d0facf42a050d3
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [5]:
import pandas as pd

# Leer los archivos de Excel
obligaciones_clientes = pd.read_excel('./datos/obligaciones_clientes.xlsx', sheet_name="Obligaciones_clientes")
tasas_productos = pd.read_excel('./datos/tasas_productos.xlsx', sheet_name="Tasas")


# Se requiere tomar las obligaciones de cada cliente y agregar la tasa correspondiente al producto asignado

In [3]:
import pandas as pd
from pandasql import sqldf

# Primero toca inicializar una variable para que se pueda ejecutar SQL en sqldf
pysqldf = lambda q: sqldf(q, globals())

# Consulta para hacer el join de las dos tablas y asignar la tasa que corresponde
# al producto de cada fila.
query = """
    SELECT
        ob.*,
        CASE
            WHEN LOWER(ob.id_producto) LIKE '%cartera%' THEN tp.tasa_cartera
            WHEN LOWER(ob.id_producto) LIKE '%hipotecario%' THEN tp.tasa_hipotecario
            WHEN LOWER(ob.id_producto) LIKE '%leasing%' THEN tp.tasa_leasing
            WHEN LOWER(ob.id_producto) LIKE '%sufi%' THEN tp.tasa_sufi
            WHEN LOWER(ob.id_producto) LIKE '%factoring%' THEN tp.tasa_factoring
            WHEN LOWER(ob.id_producto) LIKE '%tarjeta%' THEN tp.tasa_tarjeta
            WHEN LOWER(ob.id_producto) LIKE '%operacion_especifica%' THEN tp.tasa_operacion_especifica
            ELSE NULL
        END AS tasa_asignada
    FROM
        obligaciones_clientes AS ob
    LEFT JOIN
        tasas_productos AS tp
    ON
        ob.cod_segm_tasa = tp.cod_segmento AND
        ob.cod_subsegm_tasa = tp.cod_subsegmento AND
        ob.cal_interna_tasa = tp.calificacion_riesgos
"""

# Ejecutar la consulta
result_df = pysqldf(query)

print(result_df.head())


      radicado  num_documento  cod_segm_tasa  cod_subsegm_tasa  \
0  14427616502     1081648945              5                 2   
1  14427616257     1032058622              2                -1   
2  14427615471     1081374249              5                 2   
3  14426546522      996136114              5                 1   
4  14427616197     1080662762              5                 2   

  cal_interna_tasa                             id_producto  \
0               C4              OEL - operacion_especifica   
1               R1  0000000000097492172 - 29-Cartera Total   
2               C4                           OEL - leasing   
3               C1  0000000090000272791 - 29-Cartera Total   
4               C4  0000000090000242224 - 29-Cartera Total   

        tipo_id_producto  valor_inicial            fecha_desembolso  plazo  \
0    cod_plan - producto   1.050000e+06  2023-01-31 00:00:00.000000   60.0   
1  Obligacion - producto   2.848500e+05  2021-09-30 00:00:00.000000   61.0

# Se debe convertir la tasa a una tasa efectiva

In [4]:
# Desde el punto 1 ya está inicializada la variable para ejecutar las queries

# Consulta para calcular la tasa efectiva dependiendo de la periodicidad de la fila
query = """
SELECT
    *,
    POWER(1 + tasa_asignada, 1.0 / (12.0 /
        CASE
            WHEN LOWER(periodicidad) = 'mensual' THEN 1
            WHEN LOWER(periodicidad) = 'bimensual' THEN 2
            WHEN LOWER(periodicidad) = 'trimestral' THEN 3
            WHEN LOWER(periodicidad) = 'semestral' THEN 6
            WHEN LOWER(periodicidad) = 'anual' THEN 12
            ELSE NULL
        END)) - 1 AS tasa_efectiva
FROM result_df
"""

# Ejecutar la consulta
result_df = sqldf(query)

print(result_df.head())


      radicado  num_documento  cod_segm_tasa  cod_subsegm_tasa  \
0  14427616502     1081648945              5                 2   
1  14427616257     1032058622              2                -1   
2  14427615471     1081374249              5                 2   
3  14426546522      996136114              5                 1   
4  14427616197     1080662762              5                 2   

  cal_interna_tasa                             id_producto  \
0               C4              OEL - operacion_especifica   
1               R1  0000000000097492172 - 29-Cartera Total   
2               C4                           OEL - leasing   
3               C1  0000000090000272791 - 29-Cartera Total   
4               C4  0000000090000242224 - 29-Cartera Total   

        tipo_id_producto  valor_inicial            fecha_desembolso  plazo  \
0    cod_plan - producto   1.050000e+06  2023-01-31 00:00:00.000000   60.0   
1  Obligacion - producto   2.848500e+05  2021-09-30 00:00:00.000000   61.0

# Tomar la tasa efectiva, multiplicarla por el valor_inicial y dejar este resultado como valor_final, el resultado de esta tabla debe quedar almacenado.

In [7]:
# Desde el punto 1 ya está inicializada la variable para ejecutar las queries

# valor_final = tasa efectiva * valor_inicial
query = """
SELECT
    *,
    tasa_efectiva * valor_inicial AS valor_final
FROM result_df
"""

# Ejecutar la consulta SQL
final_df = sqldf(query)

# Guardar el DataFrame en un archivo Excel
final_df.to_excel("./resultados/valor_final_SQL.xlsx", index=False)

print(final_df[['valor_final']].head())


    valor_final
0  16284.871534
1   2742.580271
2    795.544732
3   9286.758274
4   2790.889388


# Se necesita sumar el valor_final de todas las obligaciones por cliente y dejar únicamente las que tenga una cantidad de productos mayor o igual a 2, el resultado de esta tabla debe quedar almacenado.

In [8]:
# Desde el punto 1 ya está inicializada la variable para ejecutar las queries

# agrupar por cliente, calcular la suma y contar productos
# luego filtrar clientes con al menos 2 productos
query = """
SELECT
    num_documento,
    SUM(valor_final) AS total_valor_final,
    COUNT(id_producto) AS cantidad_productos
FROM final_df
GROUP BY num_documento
HAVING COUNT(id_producto) >= 2
"""

# Ejecutar la consulta SQL
clientes_filtrados = sqldf(query)

# Guardar el resultado en un archivo Excel
clientes_filtrados.to_excel("./resultados/clientes_filtrados_SQL.xlsx", index=False)

print(clientes_filtrados.head())


   num_documento  total_valor_final  cantidad_productos
0        3907738        6741.584647                  10
1        4234923       18035.643930                   8
2        4935132       14118.804577                   8
3        7061608       13730.266357                   5
4        7099543       10100.998957                   6
